In [95]:
from __future__ import annotations
from typing import Union, Tuple
from sklearn.preprocessing import RobustScaler, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import lightgbm as lgb
import xgboost as xgb
from sklearn.metrics import accuracy_score, confusion_matrix
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
from torch.utils.data import TensorDataset, DataLoader
 
import sys
sys.path.append('..')
from data_processing.data_import import Elmy_import, add_date_features
from training.helper_func import clip_target, scale_target, custom_sign, threshold_percentile
from visualization.visualize import visualize_nan
from training.train2 import add_col_diff
from prediction.ensemble import ensemble_model, load_submission, ensemble_model_2

# Transform the data for a CNN

In [134]:
X_test_raw = Elmy_import('../data/raw/X_test_raw.csv', with_date=False)
y_train_raw = Elmy_import('../data/raw/y_train_raw.csv', with_date=False)
X_train_raw = Elmy_import('../data/raw/X_train_raw.csv', with_date=False)
y_test_random_raw = Elmy_import('../data/raw/y_test_random.csv', with_date=False)

In [142]:
def check_same_index(X: pd.DataFrame, y: pd.DataFrame):
    assert X.index.equals(y.index), "X and X_testy must have the same index"

def get_CNN_data_from_df(df: pd.DataFrame):
    df['date'] = pd.to_datetime(df.index, utc=True).date
    grouped_df = df.groupby('date')
    day_dict = {}
    final_dates = []
    for date, group in grouped_df:
        if group.shape[0] == 24:
            final_dates.append(date)
            day_dict[date] = group.drop('date', axis=1).T
    array_3d = np.stack([dataframe.to_numpy() for dataframe in day_dict.values()])
    return array_3d, np.array(final_dates)

def check_same_dates(dates_X, dates_y):
    assert np.all(dates_X==dates_y), "Final dates must be the same for X and y"
    

def get_CNN_data_from_X_train_X_test_no_shuffle(X_train: pd.DataFrame, y_train: pd.DataFrame, X_test: pd.DataFrame, y_test: Union[pd.DataFrame, None], train_weights: Union[pd.DataFrame, None], test_weights: Union[pd.DataFrame, None]) -> Tuple[np.ndarray, np.ndarray, np.ndarray, np.ndarray]:
    """
    Convert input DataFrames into 3D numpy arrays for use in a Convolutional Neural Network (CNN).

    This function takes in train and test DataFrames and converts them into 3D numpy arrays without
    shuffling the data. The function assumes that the input DataFrames have a date-like index and
    drops any days that do not have data for all 24 hours.

    The resulting 3D numpy arrays have shapes:
        - X_array_3d.shape = n_days x n_features x 24
        - y_array_3d.shape = n_days x 1 x 24

    Parameters
    ----------
    X_train : pd.DataFrame
        Training input features DataFrame with a date-like index.
    y_train : pd.DataFrame
        Training target DataFrame with a date-like index.
    X_test : pd.DataFrame
        Test input features DataFrame with a date-like index.
    y_test : Union[pd.DataFrame, None]
        Test target DataFrame with a date-like index, or None if not provided.

    Returns
    -------
    X_train_array_3d : np.ndarray
        3D numpy array of training input features.
    y_train_array_3d : np.ndarray
        3D numpy array of training target data.
    train_dates : np.ndarray
        1D numpy array of dates from the training data.
    X_test_array_3d : np.ndarray
        3D numpy array of test input features.
    y_test_array_3d : Union[np.ndarray, None]
        3D numpy array of test target data, or None if not provided.
    test_weights_array_3d: Union[np.ndarray, None]
        3D numpy array of test weights, or None if not provided.
    test_dates : np.ndarray
        1D numpy array of dates from the test data.

    Raises
    ------
    ValueError
        If X_train and X_test do not have the same date-like index, or if the shapes of the resulting
    numpy arrays do not match the expected format.
    """
    check_same_index(X_train, y_train)
    X_train_val_array_3d, train_val_dates = get_CNN_data_from_df(X_train)
    y_train_val_array_3d, train_dates_y = get_CNN_data_from_df(y_train)
    y_train_val_array_3d, train_dates_y = get_CNN_data_from_df(y_train)
    check_same_dates(train_val_dates, train_dates_y)
    X_test_array_3d, test_dates = get_CNN_data_from_df(X_test)
    if train_weights is not None: 
        check_same_index(X_train, train_weights)
        train_val_weights_array_3d, train_dates_weights = get_CNN_data_from_df(train_weights)
        check_same_dates(train_val_dates, train_dates_weights)
    else:
        train_val_weights_array_3d = None
    if y_test is not None:
        check_same_index(X_test, y_test)
        y_test_array_3d, test_dates_y = get_CNN_data_from_df(y_test)
        check_same_dates(test_dates, test_dates_y)
    else:
        y_test_array_3d = None
    if test_weights is not None: 
        check_same_index(X_test, test_weights)
        test_weights_array_3d, test_dates_weights = get_CNN_data_from_df(test_weights)
        check_same_dates(test_dates, test_dates_weights)
    else:
        test_weights_array_3d = None
    return X_train_val_array_3d, y_train_val_array_3d, train_val_weights_array_3d, train_val_dates, X_test_array_3d, y_test_array_3d, test_weights_array_3d, test_dates

device = 'cpu'
def get_loaders(X_train_val_array_3d, y_train_val_array_3d, train_val_weights, batch_size, train_size, num_workers, random_val_set):
    all_indices = np.arange(len(X_train_val_array_3d))
    if random_val_set:
        np.random.shuffle(all_indices)
    train_indices = all_indices[:int(train_size * len(X_train_val_array_3d))]
    val_indices = all_indices[int(train_size * len(X_train_val_array_3d)):]
    X_train_tensor = torch.tensor(X_train_val_array_3d[train_indices], dtype=torch.float32)
    y_train_tensor = torch.tensor(y_train_val_array_3d[train_indices], dtype=torch.float32)

    X_val_tensor = torch.tensor(X_train_val_array_3d[val_indices], dtype=torch.float32, device=device)
    y_val_tensor = torch.tensor(y_train_val_array_3d[val_indices], dtype=torch.float32, device=device)

    del X_train_val_array_3d, y_train_val_array_3d, all_indices

    if train_val_weights is None:
        train_weights_tensor = torch.tensor(np.ones(len(train_indices)), dtype=torch.float32)
        val_weights_tensor = torch.tensor(np.ones(len(val_indices)), dtype=torch.float32, device=device)
    else:
        train_weights_tensor = torch.tensor(train_val_weights[train_indices], dtype=torch.float32).abs()
        val_weights_tensor = torch.tensor(train_val_weights[val_indices], dtype=torch.float32, device=device).abs()
    train_dataset = TensorDataset(X_train_tensor, y_train_tensor, train_weights_tensor)
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=num_workers)
    return train_loader, val_weights_tensor, X_val_tensor, y_val_tensor, train_indices, val_indices

In [143]:
X_train_val_array_3d, y_train_val_array_3d, train_val_weights_array_3d, train_val_dates, X_test_array_3d, y_test_array_3d, test_weights_array_3d, test_dates = get_CNN_data_from_X_train_X_test_no_shuffle(X_train_raw, y_train_raw, X_test_raw, None, y_train_raw, y_test_random_raw)

X_train_val_array_3d.shape

(431, 9, 24)

In [149]:
dates = X_train_raw.date

In [141]:
len(train_val_dates)

431

In [148]:
type(X_train_raw.date)

pandas.core.series.Series

In [152]:
type(dates)

pandas.core.series.Series

In [150]:
X_train_raw[dates.isin(train_val_dates)]

,load_forecast,coal_power_available,gas_power_available,nucelear_power_available,wind_power_forecasts_average,solar_power_forecasts_average,wind_power_forecasts_std,solar_power_forecasts_std,predicted_spot_price,date
DELIVERY_START,,,,,,,,,,
2022-01-02 01:00:00+01:00,50199.0,2226.0,11487.0,43371.0,5881.0,0.0,157.781681,0.0,NaN,2022-01-02
2022-01-02 02:00:00+01:00,48690.0,2226.0,11487.0,43371.0,6110.0,0.0,85.756533,0.0,NaN,2022-01-02
2022-01-02 03:00:00+01:00,45866.0,2226.0,11487.0,43501.0,6457.0,0.0,219.432371,0.0,NaN,2022-01-02
2022-01-02 04:00:00+01:00,44697.0,2226.0,11487.0,44416.0,6856.0,0.0,286.658231,0.0,NaN,2022-01-02
2022-01-02 05:00:00+01:00,44679.0,3386.0,11487.0,44416.0,7283.0,0.0,312.730945,0.0,NaN,2022-01-02
...,...,...,...,...,...,...,...,...,...,...
2023-03-28 21:00:00+02:00,54317.0,3386.0,11952.0,36990.0,5245.0,0.0,238.896556,0.0,141.36,2023-03-28
2023-03-28 22:00:00+02:00,54269.0,3386.0,11952.0,36990.0,5356.0,0.0,141.933550,0.0,139.47,2023-03-28
2023-03-28 23:00:00+02:00,54353.0,3386.0,11952.0,36990.0,5346.0,0.0,99.820299,0.0,127.58,2023-03-28


In [55]:
test_size = 0.2

# Split the array into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X_train_array_3d, y_array_3d, test_size=test_size, shuffle=False)

# Print the shapes of the resulting arrays
print("Training data shape:", X_train.shape)
print("Validation data shape:", X_val.shape)

Training data shape: (344, 9, 24)
Validation data shape: (87, 9, 24)


In [61]:

# Convert numpy arrays to PyTorch tensors
X_train_tensor = torch.from_numpy(X_train).float()
y_train_tensor = torch.from_numpy(y_train).float()
X_val_tensor = torch.from_numpy(X_val).float()
y_val_tensor = torch.from_numpy(y_val).float()

# Create TensorDatasets for training and validation data
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
val_dataset = TensorDataset(X_val_tensor, y_val_tensor)

# Create DataLoaders
batch_size = 2
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

# Iterate over the data loaders and print the shape of each batch
print("Training data batches:")
for X_batch, y_batch in train_loader:
    print("Batch shape:", X_batch.shape)
    print("Target shape:", y_batch.shape)

Training data batches:
Batch shape: torch.Size([2, 9, 24])
Target shape: torch.Size([2, 1, 24])
Batch shape: torch.Size([2, 9, 24])
Target shape: torch.Size([2, 1, 24])
Batch shape: torch.Size([2, 9, 24])
Target shape: torch.Size([2, 1, 24])
Batch shape: torch.Size([2, 9, 24])
Target shape: torch.Size([2, 1, 24])
Batch shape: torch.Size([2, 9, 24])
Target shape: torch.Size([2, 1, 24])
Batch shape: torch.Size([2, 9, 24])
Target shape: torch.Size([2, 1, 24])
Batch shape: torch.Size([2, 9, 24])
Target shape: torch.Size([2, 1, 24])
Batch shape: torch.Size([2, 9, 24])
Target shape: torch.Size([2, 1, 24])
Batch shape: torch.Size([2, 9, 24])
Target shape: torch.Size([2, 1, 24])
Batch shape: torch.Size([2, 9, 24])
Target shape: torch.Size([2, 1, 24])
Batch shape: torch.Size([2, 9, 24])
Target shape: torch.Size([2, 1, 24])
Batch shape: torch.Size([2, 9, 24])
Target shape: torch.Size([2, 1, 24])
Batch shape: torch.Size([2, 9, 24])
Target shape: torch.Size([2, 1, 24])
Batch shape: torch.Size([2, 

# Model

In [124]:
import torch
import torch.nn as nn

class CNN(nn.Module):
    def __init__(self, num_features, input_size=24, verbose=True):
        super(CNN, self).__init__()
        self.num_features = num_features
        self.input_size = input_size
        self.conv1 = nn.Conv1d(num_features, 64, kernel_size=3)
        self.relu1 = nn.ReLU()
        self.pool1 = nn.MaxPool1d(kernel_size=2)
        self.conv2 = nn.Conv1d(64, 128, kernel_size=3)
        self.relu2 = nn.ReLU()
        self.pool2 = nn.MaxPool1d(kernel_size=2)
        self.flatten = nn.Flatten()
        input_size_fc1 = self.infer_input_size_fc1(verbose)
        self.fc1 = nn.Linear(input_size_fc1, 256)
        self.relu3 = nn.ReLU()
        self.fc2 = nn.Linear(256, 128)
        self.relu4 = nn.ReLU()
        self.fc3 = nn.Linear(128, 24)

    def infer_input_size_fc1(self, verbose):
        with torch.no_grad():
            tmp = torch.zeros((1, self.num_features, self.input_size))
            tmp = self.conv1(tmp)
            tmp = self.pool1(tmp)
            tmp = self.conv2(tmp)
            tmp = self.pool2(tmp)
            tmp = self.flatten(tmp)
            input_size_fc1 = tmp.shape[1]
            if verbose:
                print(f"Infered input size for fc1: {input_size_fc1}")
        return input_size_fc1

    def forward(self, x):
        x = self.conv1(x)
        x = self.relu1(x)
        x = self.pool1(x)
        x = self.conv2(x)
        x = self.relu2(x)
        x = self.pool2(x)
        x = self.flatten(x)
        x = self.fc1(x)
        x = self.relu3(x)
        x = self.fc2(x)
        x = self.relu4(x)
        x = self.fc3(x)
        return x


In [125]:
model = CNN(9, 24)
criterion  = nn.MSELoss()
for X_batch, y_batch in train_loader:
    y_batch = torch.squeeze(y_batch, dim=1)
    print("Batch shape:", X_batch.shape)
    print("Target shape:", y_batch.shape)
    y_pred = model(X_batch)
    print(y_pred.shape)
    loss = criterion(y_pred, y_batch)
    break

Infered input size for fc1: 512
Batch shape: torch.Size([2, 9, 24])
Target shape: torch.Size([2, 24])
torch.Size([2, 24])
